# Food Vision Project

In [ ]:
"""
We are going to exploit Mixed Precision to speed up our training process. https://www.tensorflow.org/guide/mixed_precision
In order to use mixed precision, we need compatible GPU.
"""

'\nWe are going to exploit Mixed Precision to speed up our training process. https://www.tensorflow.org/guide/mixed_precision\nIn order to use mixed precision, we need compatible GPU.\n'

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# Get the helper function
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2025-07-25 09:27:49--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-07-25 09:27:50 (46.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys
import tensorflow as tf

## Use Tensorflow Datasets to import the data

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
# List all available datasets
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

True


In [ ]:
# Load in the data
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train", "validation"],
                                             shuffle_files=True,
                                             as_supervised=True, # data gets returned in tuple (data, label)
                                             with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
ds_info.features

In [ ]:
# Get the class names
class_names = ds_info.features["label"].names
class_names[:10]

In [ ]:
# Take one sample from the train data
one_sample = train_data.take(1)
one_sample

In [ ]:
for image, label in one_sample:
  print(f"""
  image shape: {image.shape}
  image dtype: {image.dtype}
  target class from food101 dataset: {label}
  class name (from Food101 config): {class_names[label]}
  """)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)
plt.title(class_names[label])
plt.axis(False)

In [ ]:
# Create a function to preprocess our data
def preprocess_img(image, label, img_shape=224):
  """
  Converts image datatype -> float32 and reshapes image to (img_shape, img_shape, color_channels)
  """
  image = tf.image.resize(image, [img_shape, img_shape])
  return tf.cast(image, tf.float32), label

In [ ]:
preprocessed_img = preprocess_img(image, label)[0]
preprocessed_img

## Prepare the dataset

In [ ]:
# Map preprocessing function to training (and parallelize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE) # num_parallel_calls tells utilize all cpu, or gpu to compute this
# Batch the data and prefetch it (load it faster)
train_data = train_data.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE) # we already shuffled our data

test_data = test_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_data, test_data

## Create modelling callbacks

In [ ]:
from helper_functions import create_tensorboard_callback

checkpoint_path = '/content/food_vision_model.weights.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor='val_acc',
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      verbose=0) # don't print whether or not model is being saved

## Setup mixed precision training

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

## Build feature extraction model

In [ ]:
# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False
inputs = tf.keras.layers.Input(shape=input_shape, name='input_layer')
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(len(class_names))(x)
outputs = tf.keras.layers.Activation('softmax', dtype=tf.float32, name='softmax_float32')(x)
model_0 = tf.keras.Model(inputs, outputs)

# Compile the model
model_0.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model_0.summary()

## Checking layer dtype policies (are we using mixed precisions)

In [ ]:
for layer in model_0.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

## Fit the data

In [ ]:
history_0_feature_extract = model_0.fit(train_data,
                                        epochs=3,
                                        validation_data=test_data,
                                        validation_steps=int(0.10 * len(test_data)),
                                        callbacks=[create_tensorboard_callback('training_logs', 'efficient_netb0'),
                                                   model_checkpoint])

In [ ]:
# Later, You got to train finetuning model and evaluate the model and visualize the model